In [1]:
from models.user import User
from models.scheduleEntry import ScheduleEntry
from models.mapbox import MapboxOptimizer
import firebase
import random
from datetime import datetime, time, timedelta
import folium

In [2]:
firebase.delete_all_collections()

In [3]:
def display_route(route, waypoints=None):
    route_coords = [(point['lat'], point['lng']) for point in route['geometry']]

    # Center map on the first point
    m = folium.Map(location=route_coords[0])

    # Add the route as a PolyLine
    folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.8).add_to(m)

    if waypoints is not None:
        new_waypoints = [waypoints[i] for i in route["waypoint_indices"]]
        for waypoint in new_waypoints:
            folium.Marker(waypoint).add_to(m)
    else:
        folium.Marker(route_coords[0], tooltip="Start").add_to(m)
        folium.Marker(route_coords[-1], tooltip="End").add_to(m)
    # Display the map
    m.fit_bounds(route_coords)
    return m

In [4]:
def generate_random_date():
    today = datetime.now().date()
    one_week_later = today + timedelta(days=7)
    delta_days = (one_week_later - today).days
    random_days = random.randint(0, delta_days)
    return today + timedelta(days=random_days)

def generate_random_time(start_hour=6, end_hour=19):
    hour = random.randint(start_hour, end_hour-1)
    minute = random.choice([0, 15, 30, 45])
    return time(hour, minute)

In [5]:
users = []
for i in range(5):
    users.append(User(save_object=True))

In [6]:
scheduleEntries = []
for i in range(20):
    user: User = random.choice(users)
    role = random.choice(["driver", "rider"])
    date = generate_random_date()
    start_time = generate_random_time()
    arrival_delta = timedelta(minutes=random.randint(15, 90))
    arrival_datetime = datetime.combine(date, start_time) + arrival_delta
    arrival_time = arrival_datetime.time()
    max_delay = random.randint(10, 30)
    direction = "work" if start_time < time(12, 0) else "home"
    scheduleEntries.append(ScheduleEntry(user.id, role=role, date=date, start_time=start_time, arrival_time=arrival_time, max_delay=max_delay, save_object=True, direction=direction))

In [7]:
rides = firebase.get_all_rides()
#for ride in rides:
#    driver = firebase.get_user(ride.user_id)
    #print(f"Ride driver: {driver.firstName} {driver.lastName}")
print(len(rides))

9


In [13]:
firebase_users = firebase.get_all_users()
for user in firebase_users:
    schedules = firebase.get_schedule_for_user(user_id=user.id)
    print(f"Found {len(schedules)} schedule entries for {user.firstName} {user.lastName}")
    for schedule in schedules:
        print(schedule.start_time, schedule.max_delay, schedule.role)
    print("")

Found 5 schedule entries for Ali Nafzaoui
12:30:00 17 rider
10:15:00 16 rider
07:15:00 17 rider
08:15:00 22 driver
07:30:00 25 driver

Found 6 schedule entries for Omar Belhadj
10:30:00 20 driver
15:45:00 26 rider
08:15:00 24 rider
10:15:00 19 driver
07:00:00 24 rider
12:30:00 17 rider

Found 3 schedule entries for Firas Trabelsi
17:30:00 23 rider
17:00:00 18 rider
17:15:00 17 driver

Found 4 schedule entries for Sana Sayyadi
14:00:00 11 driver
09:15:00 30 driver
09:00:00 12 driver
16:30:00 14 rider

Found 2 schedule entries for Sami Stambouli
06:00:00 23 rider
12:15:00 15 driver



In [9]:
optimizer = MapboxOptimizer()

In [10]:
db = firebase.db
stihl_users = [User.from_dict(doc.to_dict()) for doc in db.collection('users').where('company', '==', 'STIHL').stream()]
for user in stihl_users:
    print(user.firstName)

Ali
Firas
Sana


In [11]:
route = optimizer.calculate_direct_route(start=users[0].home, end=users[0].work)
m = display_route(route)

In [12]:
m